# Build a Financial Analyst ReAct Agentic AI System with LangChain

### Building a Financial Analyst ReAct Agentic AI System

In this project, we will design a **Financial Analyst ReAct Agentic AI System** to assist investors by providing accurate and up-to-date stock market insights. This system will leverage the `create_react_agent` function from **LangGraph**, a more robust and modern alternative to the deprecated LangChain version. The workflow comprises the following components:

1. **Agent System Prompt**: The agent is designed to:
   - Validate input queries to ensure they are relevant and specific.
   - Provide detailed market analysis or individual stock insights depending on the user's query.
   - For invalid queries, respond professionally and guide the user appropriately.
   - Deliver concise, accurate, and professional responses while emphasizing data accuracy.

   **Flows**:
   - **Flow 1**: For general market queries, use tools to analyze trends and suggest stock opportunities.
   - **Flow 2**: For stock-specific queries, validate the ticker, retrieve data, and provide insights.

2. **Financial Analysis Tools**: The system utilizes the following tools to interact with stock market data:
   - **GET_STOCK_TICKER**: Validates and fetches stock tickers based on user queries.
   - **GET_STOCK_PRICE_METRICS**: Retrieves price trends and metrics for specific stocks.
   - **GET_STOCK_NEWS**: Extracts recent news articles related to stocks or markets.
   - **GET_STOCK_FUNDAMENTAL_INDICATOR_METRICS**: Provides insights into financial indicators like P/E ratio, ROE, etc.
   - **GET_GENERAL_MARKET_DATA**: Fetches general market trends and data.

3. **Stock Market Data Providers**: The system integrates with leading data providers such as Yahoo Finance, Finviz, Cboe, and others through platforms like __OpenBB__ to ensure reliable and real-time information.

4. **ReAct Reasoning Framework**: The agent employs ReAct reasoning to combine logical deduction with dynamic tool usage, ensuring precise and actionable results.

5. **Final Response**: After processing the data, the agent delivers a well-structured response emphasizing clarity and encouraging users to verify any investment decisions.

By utilizing LangGraph's `create_react_agent` and integrating comprehensive financial data tools, this system empowers investors with actionable insights, enabling informed decision-making in the dynamic stock market environment.

![](https://i.imgur.com/ZbYjVIE.png)

## Install OpenAI, and LangChain and OpenBB dependencies

In [ ]:
!pip install -qq langchain==0.3.14
!pip install -qq langchain-openai==0.3.0
!pip install -qq langchain-community==0.3.14
!pip install -qq langgraph==0.2.66
!pip install -qq yfinance==0.2.51

In [ ]:
pip install -q openbb

In [ ]:
!pip install -qq openbb-cboe

In [ ]:
# !pip install -qq openbb[all]

## Enter Open AI API Key

In [ ]:
# from getpass import getpass

# OPENAI_KEY = getpass('Enter Open AI API Key: ')

## Enter OpenBB Personal Access Token

In [ ]:
# from getpass import getpass

# OPENBB_PAT = getpass('Enter OpenBB Personal Access Token (PAT): ')

## Setup Environment Variables

In [ ]:
# import os

# os.environ['OPENAI_API_KEY'] = OPENAI_KEY

In [ ]:
import os

In [ ]:
OPENBB_PAT = os.getenv('OPENBB_PAT')

In [ ]:
from openbb import obb
# takes 1 min to setup everything
obb.account.login(pat=OPENBB_PAT)

## Create Financial Data Retrieval Tools

We will create custom tools as following which can be used by our Agentic AI System to interact with stock market data:

   - **GET_STOCK_TICKER**: Validates and fetches stock tickers based on user queries.
   - **GET_STOCK_PRICE_METRICS**: Retrieves price trends and metrics for specific stocks.
   - **GET_STOCK_NEWS**: Extracts recent news articles related to stocks or markets.
   - **GET_STOCK_FUNDAMENTAL_INDICATOR_METRICS**: Provides insights into financial indicators like P/E ratio, ROE, etc.
   - **GET_GENERAL_MARKET_DATA**: Fetches general market trends and data.

   
![](https://i.imgur.com/C9Ay1kv.png)

Refer to the comprehensive [OpenBB Documentation](https://docs.openbb.co/platform/reference) for understanding the specific functions used below in more detail.


In [ ]:
from langchain_core.tools import tool
from datetime import datetime, timedelta

@tool
def get_stock_ticker_symbol(stock_name: str) -> str:
    """Get the symbol, name and CIK for any publicly traded company"""
    # Use OpenBB to search for stock ticker symbol and company details by name.
    # The provider "sec" fetches data from the U.S. Securities and Exchange Commission (SEC).
    res = obb.equity.search(stock_name, provider="intrinio")

    # Convert the result to a DataFrame and format it as markdown for readability.
    stock_ticker_details = res.to_df().to_markdown()

    # Prepare the output with the stock details.
    output = """Here are the details of the company and its stock ticker symbol:\n\n""" + stock_ticker_details
    return output

@tool
def get_stock_price_metrics(stock_ticker: str) -> str:
    """Get historical stock price data, stock price quote and price performance data
       like price changes for a specific stock ticker"""

    # Fetch the latest stock price quote using "cboe" provider.
    res = obb.equity.price.quote(stock_ticker, provider='intrinio')
    price_quote = res.to_df().to_markdown()

    # Retrieve stock price performance metrics (e.g., percentage change) using "finviz" provider.
    res = obb.equity.price.performance(symbol=stock_ticker, provider='finviz')
    price_performance = res.to_df().to_markdown()

    # Fetch historical price data for the past year using "yfinance" provider.
    end_date = datetime.now()
    start_date = (end_date - timedelta(days=365)).strftime("%Y-%m-%d")
    res = obb.equity.price.historical(symbol=stock_ticker, start_date=start_date,
                                      interval='1d', provider='yfinance')
    price_historical = res.to_df().to_markdown()

    # Combine the results into a formatted output.
    output = ("""Here are the stock price metrics and data for the stock ticker symbol """ + stock_ticker + """: \n\n""" +
              "Price Quote Metrics:\n\n" + price_quote +
              "\n\nPrice Performance Metrics:\n\n" + price_performance +
              "\n\nPrice Historical Data:\n\n" + price_historical)
    return output

@tool
def get_stock_fundamental_indicator_metrics(stock_ticker: str) -> str:
    """Get fundamental indicator metrics for a specific stock ticker"""

    # Retrieve fundamental financial ratios (e.g., P/E ratio, ROE) using "fmp" provider.
    res = obb.equity.fundamental.ratios(symbol=stock_ticker, period='annual',
                                        limit=10, provider='fmp')
    fundamental_ratios = res.to_df().to_markdown()

    # Fetch additional fundamental metrics (e.g., EBITDA, revenue growth) using "yfinance" provider.
    res = obb.equity.fundamental.metrics(symbol=stock_ticker, period='annual',
                                        limit=10, provider='yfinance')
    fundamental_metrics = res.to_df().to_markdown()

    # Combine fundamental ratios and metrics into a single output.
    output = ("""Here are the fundamental indicator metrics and data for the stock ticker symbol """ + stock_ticker + """: \n\n""" +
              "Fundamental Ratios:\n\n" + fundamental_ratios +
              "\n\nFundamental Metrics:\n\n" + fundamental_metrics)
    return output

@tool
def get_stock_news(stock_ticker: str) -> str:
    """Get news article headlines for a specific stock ticker"""

    # Define the date range to fetch news (last 45 days).
    end_date = datetime.now()
    start_date = (end_date - timedelta(days=45)).strftime("%Y-%m-%d")

    # Retrieve news headlines for the stock using "tmx" provider.
    res = obb.news.company(symbol=stock_ticker, start_date=start_date, provider='tmx', limit=50)
    news = res.to_df()

    # Extract relevant columns (symbols and titles) and format as markdown.
    news = news[['symbols', 'title']].to_markdown()

    # Prepare the output with the news headlines.
    output = ("""Here are the recent news headlines for the stock ticker symbol """ + stock_ticker + """: \n\n""" + news)
    return output

@tool
def get_general_market_data() -> str:
    """Get general data and indicators for the whole stock market including,
       most actively traded stocks based on volume, top price gainers and top price losers.
       Useful when you want an overview of the market and what stocks to look at."""

    # Retrieve the most actively traded stocks using "yfinance" provider.
    res = obb.equity.discovery.active(sort='desc', provider='yfinance', limit=15)
    most_active_stocks = res.to_df().to_markdown()

    # Fetch the top price gainers using "yfinance" provider.
    res = obb.equity.discovery.gainers(sort='desc', provider='yfinance', limit=15)
    price_gainers = res.to_df().to_markdown()

    # Retrieve the top price losers using "yfinance" provider.
    res = obb.equity.discovery.losers(sort='desc', provider='yfinance', limit=15)
    price_losers = res.to_df().to_markdown()

    # Combine the market data into a single formatted output.
    output = ("""Here's some detailed information of the stock market which includes most actively traded stocks, gainers and losers:\n\n""" +
              "Most actively traded stocks:\n\n" + most_active_stocks +
              "\n\nTop price gainers:\n\n" + price_gainers +
              "\n\nTop price losers:\n\n" + price_losers)
    return output


## Test Tools

In [ ]:
# from IPython.display import display, Markdown

# response = get_general_market_data.invoke({})
# display(Markdown(response))

In [ ]:
response = get_stock_ticker_symbol.invoke('Nvidia')
display(Markdown(response))

In [ ]:
response = get_stock_price_metrics.invoke('NVDA')
display(Markdown(response))

In [ ]:
response = get_stock_fundamental_indicator_metrics.invoke('NVDA')
display(Markdown(response))

In [ ]:
response = get_stock_news.invoke('NVDA')
display(Markdown(response))

## Create Financial Analyst ReAct Agent System Prompt

In [ ]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model="gpt-4o", temperature=0)
tools = [get_stock_ticker_symbol,
         get_stock_price_metrics,
         get_stock_fundamental_indicator_metrics,
         get_stock_news,
         get_general_market_data]


tools

In [ ]:
for tool in tools:
    print(tool.name, '->', tool.description)

In [ ]:
from langchain_core.messages import SystemMessage

AGENT_PREFIX = """Role: You are an AI stock market assistant tasked with providing investors
with up-to-date, detailed information on individual stocks or advice based on general market data.

Objective: Assist data-driven stock market investors by giving accurate,
complete, but concise information relevant to their questions about individual
stocks or general advice on useful stocks based on general market data and trends.

Capabilities: You are given a number of tools as functions. Use as many tools
as needed to ensure all information provided is timely, accurate, concise,
relevant, and responsive to the user's query.

Starting Flow:
Input validation. Determine if the input is asking about a specific company
or stock ticker (Flow 2). If not, check if they are asking for general advice on potentially useful stocks
based on current market data (Flow 1). Otherwise, respond in a friendly, positive, professional tone
that you don't have information to answer as you can only provide financial advice based on market data.
For each of the flows related to valid questions use the following instructions:

Flow 1:
A. Market Analysis: If the query is valid and the user wants to get general advice on the market
or stocks worth looking into for investing, leverage the general market data tool to get relevant data.

Flow 2:
A. Symbol extraction. If the query is valid and is related to a specific company or companies,
extract the company name or ticker symbol from the question.
If a company name is given, look up the ticker symbol using a tool.
If the ticker symbol is not found based on the company, try to
correct the spelling and try again, like changing "microsfot" to "microsoft",
or broadening the search, like changing "southwest airlines" to a shorter variation
like "southwest" and increasing "limit" to 10 or more. If the company or ticker is
still unclear based on the question or conversation so far, and the results of the
symbol lookup, then ask the user to clarify which company or ticker.

B. Information retrieval. Determine what data the user is seeking on the symbol
identified. Use the appropriate tools to fetch the requested information. Only use
data obtained from the tools. You may use multiple tools in a sequence. For instance,
first determine the company's symbol, then retrieve price data using the symbol
and fundamental indicator data etc. For specific queries only retrieve data using the most relevant tool.
If detailed analysis is needed, you can call multiple tools to retrieve data first.

Response Generation Flow:
Compose Response. Analyze the retrieved data carefully and provide a comprehensive answer to the user in a clear and concise format,
in a friendly professional tone, emphasizing the data retrieved.
If the user asks for recommendations you can give some recommendations
but emphasize the user to do their own research before investing.
When generating the final response in markdown,
if there are special characters in the text, such as the dollar symbol,
ensure they are escaped properly for correct rendering e.g $25.5 should become \$25.5

Example Interaction:
User asks: "What is the PE ratio for Eli Lilly?"
Chatbot recognizes 'Eli Lilly' as a company name.
Chatbot uses symbol lookup to find the ticker for Eli Lilly, returning LLY.
Chatbot retrieves the PE ratio using the proper function with symbol LLY.
Chatbot responds: "The PE ratio for Eli Lilly (symbol: LLY) as of May 12, 2024 is 30."

Check carefully and only call the tools which are specifically named below.
Only use data obtained from these tools.
"""

SYS_PROMPT = SystemMessage(content=AGENT_PREFIX)

## Create Financial Analyst ReAct Agent

In [ ]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model="gpt-4o", temperature=0)

financial_analyst = create_react_agent(model=chatgpt,
                                       tools=tools,
                                       state_modifier=SYS_PROMPT)
financial_analyst

## Test Financial Analyst ReAct Agent

In [ ]:
from IPython.display import display, Markdown

for event in financial_analyst.stream(
    {"messages": [HumanMessage(content="Tell me some stocks I could potentially invest in")]},
    stream_mode='values' #returns full agent state with all messages including updates
):
    event["messages"][-1].pretty_print()

print('\n\nFinal Response:\n')
display(Markdown(event["messages"][-1].content))

In [ ]:
for event in financial_analyst.stream(
    {"messages": [HumanMessage(content="Tell me how is Nvidia doing right now as a company and could I potentially invest in it?")]},
    stream_mode='values' #returns full agent state with all messages including updates
):
    event["messages"][-1].pretty_print()

print('\n\nFinal Response:\n')
display(Markdown(event["messages"][-1].content))

In [ ]:
def call_financial_analyst_agent(query, verbose=False):

    for event in financial_analyst.stream(
        {"messages": [HumanMessage(content=query)]},
        stream_mode='values' #returns full agent state with all messages including updates
    ):
        if verbose:
            event["messages"][-1].pretty_print()

    print('\n\nFinal Response:\n')
    display(Markdown(event["messages"][-1].content))

In [ ]:
call_financial_analyst_agent(query="Do a comparative analysis between Nvidia and Intel and recommend which one to buy or avoid?",
                             verbose=True)

In [ ]:
call_financial_analyst_agent(query="Is Intel having any issues of late, could it still be a good buy?",
                    verbose=True)

In [ ]:
call_financial_analyst_agent(query="Give me the Market Cap, PE Ratio, Revenue per share for Microsoft",
                             verbose=False)

In [ ]:
call_financial_analyst_agent(query="Do a price analysis for Apple and give me some insights from it",
                             verbose=False)